In [1]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
import sys
import re
import glob
import suftware as su
from scipy.stats import norm
from scipy.stats import poisson

e = np.exp(1)
pi = np.pi
def pseudo_log(x,base=e):
    return np.log(x+.5)/np.log(base)

In [6]:
in_dir = '/Users/jkinney/Dropbox/15_mpathic/20_mpathic_redo/20.08.16_mpsa_raw_data'
out_dir = '20.08.16_mpsa_data'

# Get in-file
in_file = f'{in_dir}/results.brca2_9nt_lib1_rep1.txt'

# Determine name of sample
m = re.match(".*/results.(?P<locus>[^_]+)_9nt_(?P<librep>.+).txt",in_file)
locus = m.groupdict()['locus']
librep = m.groupdict()['librep']
name = f'{locus}_{librep}'

# Set parameters
min_ct_per_bc=1
min_num_barcodes=10
num_resamps=100
min_sigma=1E-2 
report_every=10
estimate_mi=True
        
# Create dict to record statistics
stats_dict = {}
        
# Load file
in_df = pd.read_csv(in_file, delimiter='\t', index_col=0)
print(f'Processing {in_file}')

# Remove extraneous columns
del in_df['mis_ct']
del in_df['lib_ct']

in_df.head()

Processing /Users/jkinney/Dropbox/15_mpathic/20_mpathic_redo/20.08.16_mpsa_raw_data/results.brca2_9nt_lib1_rep1.txt


,tot_ct,ex_ct,ss,bc
0,377,27,ACAGCGGGA,TTAGCTATCGGCTGACGTCT
1,332,5,AGCGTGTAT,CCACCCAACGCGCCGTCAGT
2,320,3286,CAGGTGAGA,TTGAGGTACACTGAACAGTC
3,312,2248,CAGGTTAGA,ACCGATCTGCCACGGCGACC
4,291,8,CAAGCCTTA,AGGGACCATCCAGTTCGCCT


In [20]:
# Marginalize by splice site
data_df = in_df.groupby('ss').sum()
data_df.reset_index(inplace=True)

# Remove indices with tot_ct < 10
ix = data_df['tot_ct'] >= 10
data_df = data_df[ix]

# Sort by tot_ct
data_df.sort_values(by='tot_ct', inplace=True, ascending=False)
data_df.reset_index(inplace=True, drop=True)

# Make sequences RNA
data_df['ss'] = [ss.replace('T','U') for ss in data_df['ss']]

# Rename columns
data_df.rename(columns={'ss':'x'}, inplace=True)

# Compute y
i_n = data_df['tot_ct']
o_n = data_df['ex_ct']
y_n = np.log10((o_n+1)/(i_n+1))
data_df['y'] = y_n

# Assign to training and test sets
data_df = data_df[['tot_ct', 'ex_ct', 'y', 'x']]

# Preview dataframe
N = len(data_df)
print(f'N: {N}')
data_df.head()

N: 30490


,tot_ct,ex_ct,y,x
0,1588,66,-1.375049,ACGGUCCAU
1,1533,118,-1.110278,AUUGCCAGG
2,1459,399,-0.562293,ACAGCGGUA
3,1414,246,-0.758059,AACGCCAGG
4,1412,60,-1.364812,ACGGCUUGG
